In [1]:
import pandas as pd
import time
from io_data.dates import Dates_Operations
from ml_models.preprocessing import Preprocessing
from ml_models.groups import GROUPS
from ml_models.postprocessing import Postprocessing
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger("prophet").setLevel(logging.ERROR)
logging.getLogger("cmdstanpy").setLevel(logging.WARNING)
logging.getLogger("cmdstanpy").propagate = False
logging.getLogger("cmdstanpy").disabled = True
import matplotlib.pyplot as plt
plt.ioff()

prefix = 'D:/работа/groups/data/'
prefix_plots = 'D:/работа/groups/TEST_PLOTS/'
prefix_output = 'D:/работа/groups/RESULTS_NEW/'

Importing plotly failed. Interactive plots will not work.


## Чтение файла с исходными фактическими данными

In [2]:
df = Preprocessing.get_data_for_forecast(filename = f'{prefix}Data_by_months.xlsx',
                  list_of_replacements = ['All 18+', 'All 14-59', 'All 10-45', 'All 14-44',
                      'All 14-54', 'All 25-49', 'All 25-54', 'All 4-45', 'All 6-54', 'W 14-44', 'W 25-59'], 
                                         column_name_with_date = 'Date')

## Определение количества месяцев для прогноза

In [3]:
start_date = pd.to_datetime(df.index[-1], format = '%d.%m.%Y') #нахождение последнего месяца в DataFrame
print('start_date ', start_date)
stop_date = '31.12.2025' # 31 декабря текущего года
forecast_periods = Dates_Operations.make_difference_in_months(start_date, stop_date)
forecast_periods

start_date  2024-12-01 00:00:00


12

## Функция для расчета

In [4]:
def main(df,
         forecast_periods,
         column_name_with_date, 
         weights_filepath, 
         error_dir = None,
         plots_dir = None, 
         save_dir = None, 
         plots: bool = False, 
         test: bool = False):
    """
        Функция для запуска прогноза ансамбля ML-моделей
        Args:
            filename: Полный путь к файлу с исходными данными
            list_of_replacements: Список из листов, находящихся в файле с исходными данными
            column_name_with_date: Название столбца с датой
            weights_filepath: Полный путь к файлу с весами для каждой модели
            plots_dir: Путь к директории, куда будут сохраняться графики для каждой модели
            save_dir: Путь к директории, куда будут сохраняться итоговые графики с прогнозами
            plots: Переменная типа bool. Если True, то графики строятся. В противном случае нет.
            test: Переменная типа bool. Если True, то тестинг проводится. В противном случае нет.
        Returns:
    """
    #Определение к какой группе относятся данные по тому или иному каналу
    group_1, group_2, group_3, group_4 = GROUPS(df).initiate_group()

    avg_forecasts = []
    
    #GROUP_1
    if not group_1.empty:
        print('', 'Результаты работы различных методов для ТВ-каналов с сезонностью и трендом', sep = '\n', end = '\n')
        avg_forecast_1 = GROUPS(group_1).process_group(forecast_periods,
                                                       column_name_with_date,
                                                       type_of_group = 'GROUP_1', 
                                                       weights_filepath = weights_filepath, 
                                                       error_dir = error_dir,
                                                       plots_dir = plots_dir, 
                                                       plots = plots, 
                                                       test = test)
        print('== ИТОГОВЫЙ РЕЗУЛЬТАТ РАБОТЫ МЕТОДОВ ДЛЯ ТВ-КАНАЛОВ С СЕЗОННОСТЬЮ И ТРЕНДОМ ==', avg_forecast_1, sep = '\n', end = '\n')
        if plots:
            Postprocessing(group_1, avg_forecast_1).get_plot(column_name_with_date, 
                                                             f'{save_dir}/Cезонность и тренд')
        avg_forecasts.append(avg_forecast_1)
        
    #GROUP_2
    if not group_2.empty:
        print('', 'Результаты работы различных методов для ТВ-каналов с трендом без сезонности', sep = '\n', end = '\n')
        avg_forecast_2 = GROUPS(group_2).process_group(forecast_periods,
                                                       column_name_with_date,
                                                       type_of_group = 'GROUP_2', 
                                                       weights_filepath = weights_filepath, 
                                                       error_dir = error_dir,
                                                       plots_dir = plots_dir, 
                                                       plots = plots, 
                                                       test = test)
        print('== ИТОГОВЫЙ РЕЗУЛЬТАТ РАБОТЫ МЕТОДОВ ДЛЯ ТВ-КАНАЛОВ С ТРЕНДОМ БЕЗ СЕЗОННОСТИ ==', avg_forecast_2, sep = '\n', end = '\n')
        if plots:
            Postprocessing(group_2, avg_forecast_2).get_plot(column_name_with_date, 
                                                             f'{save_dir}/Тренд без сезонности')
        avg_forecasts.append(avg_forecast_2)

    #GROUP_3
    if not group_3.empty:
        print('', 'Результаты работы различных методов для ТВ-каналов с сезонностью без тренда', sep = '\n', end = '\n')
        avg_forecast_3 = GROUPS(group_3).process_group(forecast_periods,
                                                       column_name_with_date,
                                                       type_of_group = 'GROUP_3', 
                                                       weights_filepath = weights_filepath, 
                                                       error_dir = error_dir,
                                                       plots_dir = plots_dir, 
                                                       plots = plots, 
                                                       test = test)
        print('== ИТОГОВЫЙ РЕЗУЛЬТАТ РАБОТЫ МЕТОДОВ ДЛЯ ТВ-КАНАЛОВ С СЕЗОННОСТЬЮ БЕЗ ТРЕНДА ==', avg_forecast_3, sep = '\n', end = '\n')
        if plots:
            Postprocessing(group_3, avg_forecast_3).get_plot(column_name_with_date, 
                                                             f'{save_dir}/Сезонность без тренда')
        avg_forecasts.append(avg_forecast_3)

    #GROUP_4
    if not group_4.empty:
        print('', 'Результаты работы различных методов для ТВ-каналов без сезонности и без тренда', sep = '\n', end = '\n')
        avg_forecast_4 = GROUPS(group_4).process_group(forecast_periods,
                                                       column_name_with_date,
                                                       type_of_group = 'GROUP_4', 
                                                       weights_filepath = weights_filepath, 
                                                       error_dir = error_dir,
                                                       plots_dir = plots_dir, 
                                                       plots = plots, 
                                                       test = test)
        print('== ИТОГОВЫЙ РЕЗУЛЬТАТ РАБОТЫ МЕТОДОВ ДЛЯ ТВ-КАНАЛОВ БЕЗ СЕЗОННОСТИ И БЕЗ ТРЕНДА ==', avg_forecast_4, sep = '\n', end ='\n')
        if plots:
            Postprocessing(group_4, avg_forecast_4).get_plot(column_name_with_date, 
                                                             f'{save_dir}/Без сезонности и без тренда')
        avg_forecasts.append(avg_forecast_4)
    general_df = Postprocessing.ensemble_of_models(df, *avg_forecasts)
    return general_df, test

# **Расчет**

In [5]:
start = time.perf_counter()

general_df, test = main(df = df,
                  forecast_periods = forecast_periods, 
                  column_name_with_date = 'Date', 
                  weights_filepath = 'ml_models/config.json', 
                  error_dir = 'D:/работа/groups/ERRORS',
                  plots_dir = 'D:/работа/groups/PLOTS_NEW/',
                  save_dir = 'D:/работа/groups/RESULT_NEW/',
                  plots = False,
                  test = True)

end = time.perf_counter()

print(f'ВРЕМЯ РАБОТЫ КОДА СОСТАВИЛО: {((end-start) / 60):0.2f} мин.')


Результаты работы различных методов для ТВ-каналов с сезонностью и трендом
РЕЗУЛЬТАТ РАБОТЫ ФУНКЦИИ DEC_WITH_TREND_YEARS
РЕЗУЛЬТАТ РАБОТЫ ФУНКЦИИ ROLLMEAN_PERIODS
РЕЗУЛЬТАТ РАБОТЫ ФУНКЦИИ SEASONDEC_PERIODS
РЕЗУЛЬТАТ РАБОТЫ ФУНКЦИИ REGR_LOG
            ЗВЕЗДА (НОВОСИБИРСК) All 18+  РЕН ТВ (ЕКАТЕРИНБУРГ) All 18+  \
Date                                                                      
2024-01-01                        3.8475                         8.2269   
2024-02-01                        3.8276                         8.7941   
2024-03-01                        3.4765                         9.0866   
2024-04-01                        3.5469                         9.2851   
2024-05-01                        3.2287                         8.4322   
2024-06-01                        3.2933                         8.9828   
2024-07-01                        3.4522                         9.3164   
2024-08-01                        3.3753                         8.8186   
2024-09-0

In [6]:
general_df

,ДОМАШНИЙ (БАРНАУЛ) All 18+,ДОМАШНИЙ (ВЛАДИВОСТОК) All 18+,ДОМАШНИЙ (ВОЛГОГРАД) All 18+,ДОМАШНИЙ (ВОРОНЕЖ) All 18+,ДОМАШНИЙ (ЕКАТЕРИНБУРГ) All 18+,ДОМАШНИЙ (ИРКУТСК) All 18+,ДОМАШНИЙ (КАЗАНЬ) All 18+,ДОМАШНИЙ (КЕМЕРОВО) All 18+,ДОМАШНИЙ (КРАСНОДАР) All 18+,ДОМАШНИЙ (КРАСНОЯРСК) All 18+,...,СТС (ЧЕЛЯБИНСК) All 6-54,СТС (ЯРОСЛАВЛЬ) All 6-54,СТС (СТАВРОПОЛЬ) All 6-54,СТС (ТВЕРЬ) All 6-54,СТС (ТОМСК) All 6-54,Ю (ЕКАТЕРИНБУРГ) W 14-44,Ю (НИЖНИЙ НОВГОРОД) W 14-44,Ю (САНКТ-ПЕТЕРБУРГ) W 14-44,ДОМАШНИЙ (ЕКАТЕРИНБУРГ) W 25-59,ДОМАШНИЙ (САНКТ-ПЕТЕРБУРГ) W 25-59
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-01,6.5636,3.7421,3.7844,3.5092,3.2081,3.8224,5.0443,3.5352,3.0693,2.8979,...,7.5892,8.3970,9.9317,6.6834,7.6303,2.1493,4.6872,1.2397,2.4750,2.3004
2024-02-01,6.6516,3.0179,3.3912,3.0684,3.2035,3.8340,5.0657,3.0981,2.7193,2.9543,...,6.4595,7.6036,10.2649,7.5863,7.8933,2.5829,4.7234,1.6396,2.3548,2.1377
2024-03-01,6.4716,2.6765,3.5114,3.0616,3.1470,3.5368,4.4655,3.4886,3.1679,2.8035,...,7.7521,7.4162,10.3791,7.1721,8.0112,2.0695,6.5284,1.5950,1.9196,2.3813
2024-04-01,6.7798,2.7064,3.5978,2.6247,3.6121,3.3298,4.9179,3.2879,3.2818,3.1552,...,7.4395,6.5805,10.4612,7.5744,8.6229,1.8972,7.6121,2.2812,2.5111,2.3439
2024-05-01,5.9226,2.7578,3.8003,2.1429,3.5716,3.1441,4.4668,3.5509,3.4519,3.0090,...,6.7281,6.9812,9.1848,6.7111,8.6269,1.2659,8.3958,1.6340,2.3757,2.1926
2024-06-01,6.9958,2.6062,4.0763,2.5111,3.0527,3.5446,5.6375,3.6548,3.5485,3.2547,...,7.0442,7.1667,7.6971,6.1254,7.6087,1.9531,7.7298,1.7911,1.7066,2.6432
2024-07-01,7.0168,2.5420,3.7160,2.6425,2.6622,3.0353,5.0780,3.2633,3.4918,3.3598,...,5.2765,6.5568,8.8191,6.8576,7.4750,1.9726,7.9081,1.7297,1.7693,2.6832
2024-08-01,7.6065,2.7571,3.7214,3.1479,3.0922,3.3524,3.9552,3.5085,3.3266,3.1091,...,6.9657,7.4617,9.1683,7.8433,8.5153,2.4864,7.5940,1.8357,1.8018,1.9385
2024-09-01,6.6943,3.0110,3.2073,3.4316,3.1807,3.1121,3.6191,3.0571,3.1200,3.3087,...,7.9784,7.3916,9.4739,6.8577,8.9227,3.0306,8.0317,1.6714,1.9104,2.2242


# **Вывод ошибок прогноза**

In [7]:
if test and forecast_periods <= 12:
    test_data =df.iloc[-forecast_periods:]
    general_MAPE = (general_df/test_data - 1) * 100
    general_MAPE = general_MAPE.reset_index()
    mean_values = general_MAPE.mean()
    # Добавляем новую строку с названием индекса "Mean"
    general_MAPE.loc['Mean'] = mean_values
    general_MAPE.to_excel(f'{prefix_output}forecast_error.xlsx')

In [8]:
general_df = general_df.reset_index()
general_df.to_excel(f'{prefix_output}forecast_output.xlsx')